In [228]:
include("choice_constructor.jl")
include("choice_types.jl")
include("choice_memhandler.jl")
include("choice_core.jl")
include("choice_parser.jl")

parseVcal (generic function with 1 method)

In [229]:
function compile(source, outname=source)
    source = source*".vcal"
    outname = outname*".axm"
    vcal = load(source)
    functions = parseVcal(vcal)
    preasm = choose(functions["main"], functions)
    create(preasm, outname)
end

compile (generic function with 2 methods)

In [230]:
compile("vibeonacci2")